In [1]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')

# Launch DPS for build_stack.py

In [2]:
import os
import geopandas
import pandas as pd
import glob
import datetime
!pip install xmltodict
import xmltodict
import sys
sys.path.append('/projects/Developer/icesat2_boreal/lib')
import ExtractUtils

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


##### To run build_stack.py across a tiled raster dataset you need a dictionary with args

s3 you need to have a vector footprint of that dataset

In [3]:
TILE_NUM = 2000

In [4]:
# ESA Worldcover 2020
BUILD_STACK_DICT = {
            #'INDEX_FN': '/projects/my-public-bucket/boreal_tiles_v003.gpkg',
            'INDEX_FN': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
            'ID_COL_NAME': 'tile_num',
            'TILE_NUM':TILE_NUM,
            'INDEX_LYR': 'boreal_tiles_v003',
            # Worldcover data is accessed via its footprint, with a 's3_path' col identifying the s3 locations of each tile
            'RASTER_NAME': 'esa_worldcover_v100_2020',
            #'COVAR_TILE_FN': '/projects/my-public-bucket/analyze_agb/footprints_v100_2020_v100_2020_map-s3.gpkg',
            'COVAR_TILE_FN': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/analyze_agb/footprints_v100_2020_v100_2020_map-s3.gpkg',
            'IN_COVAR_S3_COL': 's3_path',
            'OUTDIR': '/projects/my-public-bucket/DPS_ESA_LC',
            'NODATA_VAL': 0,
            'OUTPUT_CLIP_COG_FN':'',
            'CREDENTIALS_FN': None
        }

In [5]:
# INDEX clip shapes should be in equal area
# COVAR_TILE_FN tiles (of raster COGs) should be in 4326
args = f"\
    --in_tile_fn {BUILD_STACK_DICT['INDEX_FN']} \
    --in_tile_id_col {BUILD_STACK_DICT['ID_COL_NAME']} \
    --in_tile_num {BUILD_STACK_DICT['TILE_NUM']} \
    --tile_buffer_m 0 \
    --in_tile_layer {BUILD_STACK_DICT['INDEX_LYR']} \
    -o {BUILD_STACK_DICT['OUTDIR']} \
    --topo_off \
    --covar_src_name {BUILD_STACK_DICT['RASTER_NAME']} \
    --covar_tile_fn {BUILD_STACK_DICT['COVAR_TILE_FN']} \
    --in_covar_s3_col {BUILD_STACK_DICT['IN_COVAR_S3_COL']} \
    --input_nodata_value {BUILD_STACK_DICT['NODATA_VAL']} \
    --clip"

In [60]:
!python /projects/Developer/icesat2_boreal/lib/build_stack.py $args

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string

---Running build_stack()---

The covariate's filename(s) intersecting the 0 m bbox for tile id 2000:
 s3://esa-worldcover/v100/2020/map/ESA_WorldCover_10m_2020_v100_N51W093_Map.tif
s3://esa-worldcover/v100/2020/map/ESA_WorldCover_10m_2020_v100_N51W096_Map.tif
bbox: [4238521.999999994, 4653304.000000009, 4328521.999999994, 4743304.000000009]
Writing stack as cloud-optimized geotiff: /projects/my-public-bucket/test_build_stack/esa_worldcover_v100_2020_2000_cog.tif
Orig stack shape:		 (1, 3000, 3000)
Output resolution:		 (30, 30)
Writing img to memory...
/projects/Developer/icesat2_boreal/lib/build_stack.py:93: RasterioDeprecationWarning: Source dataset should be opened in read-only mode. Use of 

In [6]:
in_params_dict = BUILD_STACK_DICT
DPS_INPUT_TILE_NUM_LIST = []

# NOTE: 
We need to register a DPS algorithm called 'run_build_stack' before proceeding to the chunks below...

## Run a DPS job across the list

In [7]:
RUN_NAME = 'build_stack_test'

In [8]:
HLS_tindex_master_fn = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/HLS_test_redo/spring2022/HLS_tindex_master.csv'
HLS_index = pd.read_csv(HLS_tindex_master_fn)

DPS_INPUT_TILE_NUM_LIST = HLS_index['tile_num'].values
DPS_INPUT_TILE_NUM_LIST_S3 = HLS_index['s3_path'].values
print(DPS_INPUT_TILE_NUM_LIST)
print(DPS_INPUT_TILE_NUM_LIST_S3)

DPS_INPUT_TILE_NUM_LIST = DPS_INPUT_TILE_NUM_LIST[0:5]
DPS_INPUT_TILE_NUM_LIST_S3 = DPS_INPUT_TILE_NUM_LIST_S3[0:5]

print(len(DPS_INPUT_TILE_NUM_LIST_S3))

[ 4441  4476 38433 ...  3848   275     7]
['s3://maap-ops-workspace/nathanmthomas/dps_output/do_HLS_stack_3-1-2_ubuntu/master/2022/spring2022/04/13/04/15/00/257106/HLS_4441_06-01_09-15_2019_2021.tif'
 's3://maap-ops-workspace/nathanmthomas/dps_output/do_HLS_stack_3-1-2_ubuntu/master/2022/spring2022/04/13/03/51/26/317765/HLS_4476_06-01_09-15_2019_2021.tif'
 's3://maap-ops-workspace/nathanmthomas/dps_output/do_HLS_stack_3-1-2_ubuntu/master/2022/spring2022/04/13/03/49/47/028016/HLS_38433_06-01_09-15_2019_2021.tif'
 ...
 's3://maap-ops-workspace/nathanmthomas/dps_output/do_HLS_stack_3-1-2_ubuntu/master/2022/spring2022/03/24/20/44/50/862412/HLS_3848_06-01_09-15_2019_2021.tif'
 's3://maap-ops-workspace/nathanmthomas/dps_output/do_HLS_stack_3-1-2_ubuntu/master/2022/spring2022/03/24/20/43/54/290821/HLS_275_06-01_09-15_2019_2021.tif'
 's3://maap-ops-workspace/nathanmthomas/dps_output/do_HLS_stack_3-1-2_ubuntu/master/2022/spring2022/03/24/20/38/59/840829/HLS_7_06-01_09-15_2019_2021.tif']
5


In [ ]:
in_params_dict = {
            #'INDEX_FN': '/projects/my-public-bucket/boreal_tiles_v003.gpkg',
            'INDEX_FN': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
            'ID_COL_NAME': 'tile_num',
            'TILE_NUM':None,
            'BUFFER': 0,
            'INDEX_LYR': 'boreal_tiles_v003',
            'OUTDIR': '/projects/my-public-bucket/DPS_ESA_LC',
            'TOPO_OFF': '',
            'RASTER_NAME': 'esa_worldcover_v100_2020',
            'COVAR_TILE_FN': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/analyze_agb/footprints_v100_2020_v100_2020_map-s3.gpkg',
            'IN_COVAR_S3_COL': 's3_path',
            'NODATA_VAL': 0,
            'OUTPUT_CLIP_COG_FN':'',
        }

In [16]:
%%time
submit_results_df_list = []
len_input_list = len(DPS_INPUT_TILE_NUM_LIST)
print(f"# of input tiles for DPS: {len_input_list}")

for i, INPUT_TILE_NUM in enumerate(DPS_INPUT_TILE_NUM_LIST):
    
    DPS_num = i+1
    IDENTIFIER = 'run_build_stack'
    ALGO_ID = f'{IDENTIFIER}_ubuntu'
    USER = 'nathanmthomas'
    WORKER_TYPE = 'maap-dps-worker-8gb'
    
    in_params_dict['TILE_NUM'] = INPUT_TILE_NUM
    print(in_params_dict)
    submit_result = maap.submitJob(
            identifier=IDENTIFIER,
            algo_id = 'build_ESA_LC_stack',
            #algo_id=ALGO_ID,
            version='master',
            username=USER, # username needs to be the same as whoever created the workspace
            queue=WORKER_TYPE,
            **in_params_dict
        )
    
    print(submit_result)
    '''
    # Build a dataframe of submission details
    submit_result['dps_num'] = DPS_num
    submit_result['tile_num'] = INPUT_TILE_NUM
    submit_result['submit_time'] = datetime.datetime.now()
    submit_result['dbs_job_hour'] =datetime.datetime.now().hour
    submit_result['algo_id'] = ALGO_ID
    submit_result['user'] = USER
    submit_result['worker_type'] = WORKER_TYPE

    # Append to a list of data frames of submission results
    submit_results_df_list.append(pd.DataFrame([submit_result]))
    
    if DPS_num in [1, 25,50, 100,200,300,400,500, 750, 1000, 1250, 1500, 1750, 2000, 2250, 2500, 2750, 3000, 3250, 3500, 3750, 4000, 4250, 5000, 7000, 9000, 11000, 13000, 15000, 17000, 19000, 21000, 24000, len_input_list]:
        print(f"DPS run #: {DPS_num}\t| tile num: {INPUT_TILE_NUM}\t| submit status: {submit_result['status']}\t| job id: {submit_result['job_id']}") 
        
# Build a final submission results df and save
submit_results_df = pd.concat(submit_results_df_list)
submit_results_df['run_name'] = RUN_NAME
nowtime = pd.Timestamp.now().strftime('%Y%m%d%H%M')
print(f"Current time:\t{nowtime}")
submit_results_df.to_csv(f'/projects/my-public-bucket/dps_submission_results/DPS_{IDENTIFIER}_submission_results_{len_input_list}_{nowtime}.csv')
submit_results_df
'''

# of input tiles for DPS: 5
{'INDEX_FN': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg', 'ID_COL_NAME': 'tile_num', 'TILE_NUM': 4536, 'INDEX_LYR': 'boreal_tiles_v003', 'RASTER_NAME': 'esa_worldcover_v100_2020', 'COVAR_TILE_FN': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/analyze_agb/footprints_v100_2020_v100_2020_map-s3.gpkg', 'IN_COVAR_S3_COL': 's3_path', 'OUTDIR': '/projects/my-public-bucket/DPS_ESA_LC', 'NODATA_VAL': 0, 'OUTPUT_CLIP_COG_FN': '', 'CREDENTIALS_FN': None, 'in_tile_num': 4441}
{'status': 'failed', 'http_status_code': 500, 'job_id': ''}
{'INDEX_FN': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg', 'ID_COL_NAME': 'tile_num', 'TILE_NUM': 4536, 'INDEX_LYR': 'boreal_tiles_v003', 'RASTER_NAME': 'esa_worldcover_v100_2020', 'COVAR_TILE_FN': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/analyze_agb/footprints_v100_2020_v100_2020_map-s3.gpkg', 'IN_COVAR_S

After almost any DPS job, you have to assess what succeeded and failed. This involves:
1. building a table of job status based on job ids captured in the job_results_df from the DPS run chunk (this takes 40 mins for ~47k jobs) --> this tells you how many jobs failed
2. merging the job status table with the job results df --> this tells you which specific granules (or tile nums) failed
3. building another input list of granules for a follow-up DPS
## Assess DPS results
Build a table of job status based on job id - how many jobs failed?

In [ ]:
%%time
LIST_SUBMISSIONS = sorted(glob.glob(f'/projects/my-public-bucket/dps_submission_results/DPS_{IDENTIFIER}_submission_results_*.csv'),key=ExtractUtils.func, reverse=True)
for DPS_DATETIME in [nowtime]:
    for fn in LIST_SUBMISSIONS:
        if DPS_DATETIME in fn and not 'job_status' in fn:
            DPS_alg_id = os.path.basename(fn.split('_submission_results_')[0].replace('DPS_',''))
            thentime = fn.split('_')[-1].replace('.csv','')
            print(f'DPS alg:\t\t{DPS_alg_id}')
            print(f'DPS launch time:\t{thentime}')
            z = ExtractUtils.BUILD_TABLE_JOBSTATUS(pd.read_csv(fn))
            # Save job status table
            z.to_csv(f'/projects/my-public-bucket/dps_submission_results/DPS_{IDENTIFIER}_submission_results_job_status_{len(z)}_{thentime}.csv')